# 1. Collect Data

In [29]:
class Note:
    def __init__(self,s,i,v):
        self.start = s
        self.relative_interval = i
        self.duration = v #duration


class Accompany:
    def __init__(self,num=4,den=4,notelist=[],classjson=None):
        if classjson is None:
            self.numerator = num
            self.denominator = den
            self.notelist = notelist
        else:
            self.numerator = classjson["numerator"]
            self.denominator = classjson["denominator"]
            self.notelist = classjson['notes']
    
    def export_dict(self):
        tmp = dict()
        tmp['numerator'] = self.numerator
        tmp['denominator'] = self.denominator
        tmp['notes'] = []
        for notes in self.notelist:
            tmp2 = dict()
            tmp2['s'] = notes.start
            tmp2['i'] = notes.relative_interval
            tmp2['d'] = notes.duration
            tmp['notes'].append(tmp2)
        return tmp

    def add_notes(self,n:Note):
        self.notelist.append(n)

In [30]:
def notes_bar_processing(notes,tpb,num,den):
    #find min pitch
    min_pitch = 128
    min_time = 1e30
    for note in notes:
        if note.pitch < min_pitch:
            min_pitch = note.pitch
        if note.start < min_time:
            min_time = note.start
    accom = Accompany(num=num,den=den,notelist=[])
    for note in notes:
        start = (note.start-min_time)/tpb
        rpitch = note.pitch - min_pitch
        dur = (note.end-note.start)/tpb
        accom.add_notes(Note(start,rpitch,dur))
    return accom
    #then just construct everything from ehre
    #note.start
    #note.end
    #note.pitch
    

In [35]:
import glob
from miditoolkit.midi import parser as mid_parser  
from miditoolkit.midi import containers as ct


database = []
for midifile in glob.glob("../data/nice_format/*.mid"): #Replace it with your own directory
    mido = mid_parser.MidiFile(midifile)
    # print(mido.time_signature_changes)
    if len(mido.time_signature_changes) > 1:
        print(f"{midifile} has more than one time signature change hence it will not be processed for now.")
        #TODO: how to deal with time siganture changes?
        # print(mido.time_signature_changes)
        # print(mido.ticks_per_beat)
        continue
    tpb = mido.ticks_per_beat
    numerator = mido.time_signature_changes[0].numerator
    denominator = mido.time_signature_changes[0].denominator
    # print(tpb)
    idx = -1
    for i,inst in enumerate(mido.instruments):
        if inst.name.find != -1:
            idx = i
            break
    if idx == -1:
        print(f"{midifile} may not have left channel, please check.")
        continue
    add_interval = tpb*numerator
    current_tick = tpb*numerator     
    notelist = []
    tmp_notelist = []
    for note in mido.instruments[idx].notes:
        if note.start < current_tick:
            if note.end > current_tick:
                tmp_notelist.append(ct.Note(start=current_tick,end=note.end,pitch=note.pitch,velocity=note.velocity))
                notelist.append(ct.Note(start=note.start,end=current_tick,pitch=note.pitch,velocity=note.velocity))
            else:
                notelist.append(note)
        else:
            database.append(notes_bar_processing(notelist,tpb,numerator,denominator))
            notelist = []
            current_tick += add_interval
            tmp2 = []
            for note2 in tmp_notelist:
                if note2.end > current_tick:
                    tmp2.append(ct.Note(start=current_tick,end=note2.end,pitch=note2.pitch,velocity=note2.velocity))
                    notelist.append(ct.Note(start=note2.start,end=current_tick,pitch=note2.pitch,velocity=note2.velocity))
                else:
                    notelist.append(note2)
            tmp_notelist = tmp2
            #add here the last bars
    database.append(notes_bar_processing(notelist,tpb,numerator,denominator))

print(len(database))
print(database[0].export_dict())
#continue build

../data/nice_format\alb_esp3.mid has more than one time signature change hence it will not be processed for now.
../data/nice_format\alb_esp5.mid has more than one time signature change hence it will not be processed for now.
../data/nice_format\beethoven_hammerklavier_2.mid has more than one time signature change hence it will not be processed for now.
../data/nice_format\beethoven_hammerklavier_4.mid has more than one time signature change hence it will not be processed for now.
../data/nice_format\beethoven_les_adieux_1.mid has more than one time signature change hence it will not be processed for now.
../data/nice_format\beethoven_opus10_3.mid has more than one time signature change hence it will not be processed for now.
../data/nice_format\bor_ps5.mid has more than one time signature change hence it will not be processed for now.
../data/nice_format\brahms_opus1_2.mid has more than one time signature change hence it will not be processed for now.
../data/nice_format\brahms_opus1_

In [36]:
frequency = dict()
for records in database:
    ks = str(records.numerator) +"/" + str(records.denominator)
    if ks in frequency:
        frequency[ks] += 1
    else:
        frequency[ks] = 1
frequency

{'3/4': 9243,
 '2/4': 7038,
 '5/8': 57,
 '3/8': 1806,
 '6/8': 1573,
 '12/8': 132,
 '4/4': 9041,
 '4/8': 285,
 '9/8': 141,
 '2/8': 46,
 '6/4': 250,
 '2/2': 153,
 '6/16': 32,
 '3/16': 92,
 '8/4': 142}

In [32]:
#skyline_reverse plus choose all notes with lowest channel for the rhythm
